In [1]:
%matplotlib inline

# Import all python modules. You need to run this cell for the other cells to work.
import requests 
import numpy as np
import pandas as pd
from datetime import datetime
import itertools
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
# Select different variables

api = 'http://api.cal-adapt.org/api' # Cal-Adapt API
resource = 'counties'
climvar = ['tasmax', 'relhumid-min'] # you can add any other climate variables such as 'tasmin', 'pr' etc
period = ['day'] # This will give you daily data
models = ['CanESM2', 'CNRM-CM5', 'HadGEM2-ES', 'MIROC5'] #Models that you want to use. 
scenarios = ['historical', 'rcp85', ] # or rcp45
#city_names = ['Sac', 'SD', 'LA', 'SB', 'Fresno', 'SJ']                                                       
points = ['POINT(-121.4944 38.5816)', 'POINT(-117.1611 32.7157)','POINT(-118.2437 34.0522)', 'POINT(-119.6982 34.4208)', 'POINT(-119.7871 36.7378)', 'POINT(-121.8863 37.3382)' ]

In [3]:
def process_daily_data(slug, city_id, city_name):
    # Make request
    url = api + '/series/' + slug + '/events/?ref=/api/' + resource + '/%s/&imperial=TRUE&stat=mean' % city_id
    response = requests.get(url)
    #print(response)
    # Get data
    if response.ok:
        data = response.json()
        df = pd.DataFrame(data)
        df.set_index('index', inplace = True)
        df.index = pd.to_datetime(df.index)
        df['city'] = city_name
    else:
        print('Failed to download:')
    return df

In [4]:
def get_projections(city_id, city_name):
    # Create an empty list to hold dataframes
    df_list = []
    # Make a combined list of GCMs, scenarios, climate variables for looping
    zipped = itertools.product(climvar, period, models, scenarios)
    # Loop through zipped
    for item in zipped:
        # Create slug
        slug = '_'.join(item)
        df = process_daily_data(slug, city_id, city_name)
        df_list.append(df)
            
    # Combine all the dataframes into one and return
    return pd.concat(df_list)

In [5]:
def projection_for_all_cities():
    df_all_cities = []
    # Cal-Adapt API
    api = 'http://api.cal-adapt.org/api'
    # Name of boundary layer in API (Census Tracts)
    #Get ID
    for i in points:
        url = api + '/' + resource + '/?intersects=' +  i
        response = requests.get(url)
        county = response.json()
        county = county['features'][0]
        city_id  = county['id']
        county_name = county['properties']['name']
        df = get_projections(city_id, county_name)
        df_all_cities.append(df)
    return pd.concat(df_all_cities)

In [6]:
df = projection_for_all_cities()
df.head()

,name,data,city
index,,,
1950-01-01 00:00:00+00:00,tasmax_day_CanESM2_historical,58.407066,Sacramento
1950-01-02 00:00:00+00:00,tasmax_day_CanESM2_historical,61.634686,Sacramento
1950-01-03 00:00:00+00:00,tasmax_day_CanESM2_historical,56.477486,Sacramento
1950-01-04 00:00:00+00:00,tasmax_day_CanESM2_historical,52.621391,Sacramento
1950-01-05 00:00:00+00:00,tasmax_day_CanESM2_historical,48.599957,Sacramento


In [7]:
# Keep only required parameters in slug
df['Climate Variable'] = df['name'].str.split('_', expand=True)[0] 
df['Model'] = df['name'].str.split('_', expand=True)[2] 
df.index.name = 'Date'
df.drop(columns= 'name', inplace = True)
df.head()

,data,city,Climate Variable,Model
Date,,,,
1950-01-01 00:00:00+00:00,58.407066,Sacramento,tasmax,CanESM2
1950-01-02 00:00:00+00:00,61.634686,Sacramento,tasmax,CanESM2
1950-01-03 00:00:00+00:00,56.477486,Sacramento,tasmax,CanESM2
1950-01-04 00:00:00+00:00,52.621391,Sacramento,tasmax,CanESM2
1950-01-05 00:00:00+00:00,48.599957,Sacramento,tasmax,CanESM2


In [8]:
df = df.pivot_table( values = 'data', index = df.index, columns = ['Climate Variable', 'Model', 'city'])

In [9]:
df.head()

Climate Variable          relhumid-min                                    \
Model                         CNRM-CM5                                     
city                            Fresno Los Angeles Sacramento  San Diego   
Date                                                                       
1950-01-01 00:00:00+00:00    48.636566   50.464371  54.745319  65.115051   
1950-01-02 00:00:00+00:00    56.282787   58.674809  60.657166  63.786407   
1950-01-03 00:00:00+00:00    53.853527   64.886566  59.479286  69.547737   
1950-01-04 00:00:00+00:00    52.999134   58.980434  60.190807  63.292316   
1950-01-05 00:00:00+00:00    49.431328   48.489601  64.596107  61.679836   

Climate Variable                                                            \
Model                                                  CanESM2               
city                      Santa Barbara Santa Clara     Fresno Los Angeles   
Date                                                                         
1950-01-01 00:00:00+00:00     47.019382   48.026974  73.742569   59.465855   
1950-01-02 00:00:00+00:00     51.946590   54.667034  71.206413   57.444157   
1950-01-03 00:00:00+00:00     61.476654   62.804329  62.935143   44.445732   
1950-01-04 00:00:00+00:00     56.478550   59.533405  59.341007   29.678896   
1950-01-05 00:00:00+00:00     49.374092   56.338585  44.416157   24.602190   

Climate Variable                                 ...     tasmax             \
Model                                            ... HadGEM2-ES              
city                      Sacramento  San Diego  ... Sacramento  San Diego   
Date                                             ...                         
1950-01-01 00:00:00+00:00  96.752136  55.021534  ...  49.986374  61.099926   
1950-01-02 00:00:00+00:00  86.312256  52.202526  ...  44.567371  61.963177   
1950-01-03 00:00:00+00:00  89.525734  45.641850  ...  46.110786  64.150719   
1950-01-04 00:00:00+00:00  82.996468  34.063301  ...  46.973488  63.756752   
1950-01-05 00:00:00+00:00  62.749279  28.718233  ...  46.166870  65.302200   

Climate Variable                                                            \
Model                                                   MIROC5               
city                      Santa Barbara Santa Clara     Fresno Los Angeles   
Date                                                                         
1950-01-01 00:00:00+00:00     61.420509   49.352409  52.391338   66.171928   
1950-01-02 00:00:00+00:00     61.833870   49.884369  53.051727   65.240448   
1950-01-03 00:00:00+00:00     61.249947   50.355408  52.499771   63.466656   
1950-01-04 00:00:00+00:00     60.488205   50.568871  51.777695   67.844757   
1950-01-05 00:00:00+00:00     60.565441   51.591370  51.568901   66.270752   

Climate Variable                                                           
Model                                                                      
city                      Sacramento  San Diego Santa Barbara Santa Clara  
Date                                                                       
1950-01-01 00:00:00+00:00  57.548046  67.757141     68.760422   58.912163  
1950-01-02 00:00:00+00:00  58.211784  66.521622     68.525803   59.519981  
1950-01-03 00:00:00+00:00  57.473503  66.155838     66.312332   62.505627  
1950-01-04 00:00:00+00:00  59.445274  69.994621     69.794670   59.723724  
1950-01-05 00:00:00+00:00  57.317665  68.880280     67.350540   58.228706  

[5 rows x 48 columns]

In [10]:
# Directory where to save the data. Don't forget to add .csv at the end of file name 
directory = r'P:\Wind Speed LOCA\County Data.csv'
csv_file = df.to_csv(directory, header=True) 

In [11]:
# To filter data through multiple headeres
df.xs(('tasmax', 'CanESM2'), axis = 1).head()

city,Fresno,Los Angeles,Sacramento,San Diego,Santa Barbara,Santa Clara
Date,,,,,,
1950-01-01 00:00:00+00:00,50.725479,57.165611,58.407066,63.673199,60.396469,58.072533
1950-01-02 00:00:00+00:00,56.098949,63.308453,61.634686,69.844109,61.427868,60.008984
1950-01-03 00:00:00+00:00,54.248634,69.373070,56.477486,76.096817,67.632996,55.467072
1950-01-04 00:00:00+00:00,50.584305,71.154831,52.621391,76.927765,68.335846,55.926796
1950-01-05 00:00:00+00:00,42.205528,67.097801,48.599957,76.221176,63.136517,48.801224


## Manually get data for different locations and GCMs

In [12]:
# Every county has an ID, that can be foun here: 'https://api.cal-adapt.org/api/counties/'
# Or you can write below code to find that ID for your location. 
# Point that is provided below is from Fresno. Code will return the ID of a county where this points belong to
# Now if you specify 10 in above URl, like this 'https://api.cal-adapt.org/api/counties/10/' it will return county Fresno

In [13]:
url = api + '/' + resource + '/?intersects=' +  'POINT(-119.7871 36.7378)'
response = requests.get(url)
county = response.json()
county = county['features'][0]
city_id  = county['id']

In [15]:
name = county['properties']['name']

In [16]:
name

'Fresno'

In [17]:
city_id

10

In [18]:
# Now you specify the id number in url to get data 
# You can modify slug according to your needs

In [19]:
slug = 'tasmax_day_CanESM2_historical'

url = api + '/series/' + slug + '/events/?ref=/api/' + resource + '/%s/&imperial=TRUE&stat=mean' % city_id
response = requests.get(url)
    #print(response)
    # Get data
if response.ok:
    data = response.json()
    df2 = pd.DataFrame(data)
    df2.set_index('index', inplace = True)
    df2.index = pd.to_datetime(df2.index)
df2.head()

,name,data
index,,
1950-01-01 00:00:00+00:00,tasmax_day_CanESM2_historical,50.725479
1950-01-02 00:00:00+00:00,tasmax_day_CanESM2_historical,56.098949
1950-01-03 00:00:00+00:00,tasmax_day_CanESM2_historical,54.248634
1950-01-04 00:00:00+00:00,tasmax_day_CanESM2_historical,50.584305
1950-01-05 00:00:00+00:00,tasmax_day_CanESM2_historical,42.205528
